## Importing necessary libraries 
- pandas : Used for data manipulation and analysis using DataFrames.
- re: Provides regular expression operations for pattern matching in text.
- TextBlob: A library for processing textual data, mainly used for sentiment analysis and NLP.
- symspellpy: A fast spelling correction library using Symmetric Delete algorithm.
- SpellChecker: Provides simple spell checking and correction for English words.
- nltk: Natural Language Toolkit, used for working with human language data (text).
- nltk.corpus import words: A list of valid English words from the NLTK corpus.
- nltk.corpus import stopwords: Provides a list of common stopwords (e.g., "the", "is") for text cleaning.


In [1]:
import pandas as pd
import re
from textblob import TextBlob
from symspellpy import SymSpell, Verbosity
from spellchecker import SpellChecker
import nltk
from nltk.corpus import words as nltk_words
from nltk.corpus import words
from nltk.corpus import stopwords

## Loading the dataset

In [2]:
# Correct absolute path
df = pd.read_csv("/Users/prarthana/Downloads/Reviews.csv")

# Preview the first 5 rows
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,1303862400,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,1346976000,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,1219017600,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,1307923200,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,1350777600,Great taffy,Great taffy at a great price. There was a wid...


In [3]:
print("Shape of dataset (rows, columns):", df.shape)

Shape of dataset (rows, columns): (568454, 10)


## Data Preprocessing
### Step 1: Checking data types of columns

In [4]:
print(df.dtypes)

Id                         int64
ProductId                 object
UserId                    object
ProfileName               object
HelpfulnessNumerator       int64
HelpfulnessDenominator     int64
Score                      int64
Time                       int64
Summary                   object
Text                      object
dtype: object


### Step 2: Changing data types where required
Since ProductId and UserId are unique identifiers for products and users, converting them to categorical and Time to datetime 

Note: These columns are not going to be used as only summary and text columns will be required for this analysis, but following the cleaning steps for ease of use for further analysis on the same data, done on a separate file.

In [5]:
# Convert 'ProductId' and 'UserId' to categorical data type
df['ProductId'] = df['ProductId'].astype('category')
df['UserId'] = df['UserId'].astype('category')

# Convert 'Time' from Unix timestamp to datetime
df['Time'] = pd.to_datetime(df['Time'], unit='s')

In [6]:
df.head()

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
0,1,B001E4KFG0,A3SGXH7AUHU8GW,delmartian,1,1,5,2011-04-27,Good Quality Dog Food,I have bought several of the Vitality canned d...
1,2,B00813GRG4,A1D87F6ZCVE5NK,dll pa,0,0,1,2012-09-07,Not as Advertised,Product arrived labeled as Jumbo Salted Peanut...
2,3,B000LQOCH0,ABXLMWJIXXAIN,"Natalia Corres ""Natalia Corres""",1,1,4,2008-08-18,"""Delight"" says it all",This is a confection that has been around a fe...
3,4,B000UA0QIQ,A395BORC6FGVXV,Karl,3,3,2,2011-06-13,Cough Medicine,If you are looking for the secret ingredient i...
4,5,B006K2ZZ7K,A1UQRSCLF8GW1T,"Michael D. Bigham ""M. Wassir""",0,0,5,2012-10-21,Great taffy,Great taffy at a great price. There was a wid...


### Step 3: Checking for null values

Here since the null values are a fairly insignificant number and present in profile name and summary, we move forward without treating it.

In [7]:
print(df.isnull().sum())

Id                         0
ProductId                  0
UserId                     0
ProfileName               26
HelpfulnessNumerator       0
HelpfulnessDenominator     0
Score                      0
Time                       0
Summary                   27
Text                       0
dtype: int64


In [9]:
num_null_rows = df.isnull().any(axis=1).sum()
print("Number of rows with at least one null value:", num_null_rows)

Number of rows with at least one null value: 53


### Step 4: Checking for duplicates

In [8]:
unique_counts = df.nunique()

# Print the result
unique_counts

Id                        568454
ProductId                  74258
UserId                    256059
ProfileName               218415
HelpfulnessNumerator         231
HelpfulnessDenominator       234
Score                          5
Time                        3168
Summary                   295742
Text                      393579
dtype: int64

In [10]:
df[df.isnull().any(axis=1)].head(10)

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
10616,10617,B002DHN956,A1LSYR30XW7CFT,NaN,1,2,5,2010-12-07,Awesome,This is 72 for the price of like 24 at the gro...
25509,25510,B000LKZB4Y,A36BVYD0NT7Z0F,NaN,0,0,5,2011-08-29,These are the best mints and no aspartame or BHT,I was so shocked to find out that almost all g...
33958,33959,B00412W76S,A3TJPSWY2HE4BS,"S. Layton ""homeschool blogger""",1,24,2,2007-03-08,NaN,I only used two maybe three tea bags and got p...
38874,38875,B000AYDGZ2,A36BVYD0NT7Z0F,NaN,2,3,1,2010-07-06,doesn't anyone care that they are putting BHT ...,I called Kellogg's to see why Special K red be...
40548,40549,B00020HHRW,A3TJPSWY2HE4BS,"S. Layton ""homeschool blogger""",1,24,2,2007-03-08,NaN,I only used two maybe three tea bags and got p...
47923,47924,B004SRH2B6,A2DEHJJIEAPPBF,NaN,0,0,5,2012-06-06,Great taste,Enjoy drinking this brand. Tastes as good as ...
49800,49801,B000CRHQN0,A2LYFY32LXQDON,NaN,0,0,2,2010-08-24,They were melted and the chocolate had turned ...,We love these bars but i won't order them ship...
67077,67078,B0006348H2,A2P0P67Y55SNOX,NaN,1,1,5,2011-08-30,Wheatgrass,Kitty seems to like this sprinkled on her food...
101106,101107,B0014B0HWK,A3TJPSWY2HE4BS,"S. Layton ""homeschool blogger""",1,24,2,2007-03-08,NaN,I only used two maybe three tea bags and got p...
102979,102980,B000FVDWU4,A3TJPSWY2HE4BS,"S. Layton ""homeschool blogger""",1,24,2,2007-03-08,NaN,I only used two maybe three tea bags and got p...


In [11]:
duplicate_count = df.duplicated().sum()
print("Number of duplicate rows:", duplicate_count)

Number of duplicate rows: 0


In [12]:
# Define the columns to check for duplicates
columns_to_check = ['ProductId', 'UserId', 'ProfileName', 'Summary', 'Text']

# Count duplicates based on those columns
dup_count = df.duplicated(subset=columns_to_check).sum()

print(f"Number of duplicate rows based on {columns_to_check}: {dup_count}")

Number of duplicate rows based on ['ProductId', 'UserId', 'ProfileName', 'Summary', 'Text']: 898


In [13]:
# List of specific columns to check for duplicates
columns_to_check = ['ProductId', 'UserId', 'ProfileName', 'Summary', 'Text']

# Loop through each column and count duplicates
for col in columns_to_check:
    dup_count = df.duplicated(subset=[col]).sum()
    print(f"Duplicates in '{col}': {dup_count}")


Duplicates in 'ProductId': 494196
Duplicates in 'UserId': 312395
Duplicates in 'ProfileName': 350038
Duplicates in 'Summary': 272711
Duplicates in 'Text': 174875


### Step 5: Treating Duplicates

Checking for duplicates in text column, there is a significant number of repeated reviews. Since we are going to use this for sentiment analysis, we want to get rid of redundant rows.

In [14]:
# Count rows where the 'Text' column is duplicated (entire 'Text' value is the same)
dup_text_count = df['Text'].duplicated(keep=False).sum()

print(f"Number of rows with duplicate 'Text' values: {dup_text_count}")

Number of rows with duplicate 'Text' values: 232915


#### Note: 
Grouping the duplicated text to identify why they are not being flagged. Here many duplicated texts have different ProductId but same UserId, Summary, and Time. If there was further information on the product we could have identified if it was for different products but here we are assuming it is repeated reviews by the same user for the product being falsely classified as different ProductsIds.

In [15]:
# Sort by 'Text' so that rows with the same 'Text' are grouped together
dup_text_rows_sorted = df[df['Text'].duplicated(keep=False)].sort_values(by='Text')

# Get the first 10 rows where the 'Text' column is duplicated and sorted by 'Text'
dup_text_rows_head = dup_text_rows_sorted.head(10)

# Print the first 10 rows with duplicated 'Text'
dup_text_rows_head

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
257785,257786,B000KOWR8E,A142S4ZZF1FJ1X,Joseph E Brew,2,3,4,2010-10-09,Better Sweetener!,"""4C Totally Light"" is one of the very few ""sug..."
506745,506746,B000KOWR8Y,A142S4ZZF1FJ1X,Joseph E Brew,0,0,4,2010-10-09,4C Totally Light,"""4C Totally Light"" is one of the very few ""sug..."
107704,107705,B001F0RRTQ,A1R7E82MN0S8V3,DENNIS,0,0,5,2012-06-12,GREAT DOG TREAT,"""BUFFY"" LOOKS FORWARD TO HER ""TOY"" EVERY AFTER..."
418609,418610,B001F0RRU0,A1R7E82MN0S8V3,DENNIS,0,0,5,2012-06-12,GREAT DOG TREAT,"""BUFFY"" LOOKS FORWARD TO HER ""TOY"" EVERY AFTER..."
561246,561247,B001JU81ZG,A7FNPP1SMY97G,D. Hsu,4,6,1,2011-11-08,Buy this if you have NO taste buds!,"""Blends smooth and creamy for a sweet tasting ..."
330089,330090,B001OHX1ZY,A7FNPP1SMY97G,D. Hsu,4,6,1,2011-11-08,Buy this if you have NO taste buds!,"""Blends smooth and creamy for a sweet tasting ..."
233264,233265,B007TJGZ4A,A17950SQVNAVOD,Scott,2,2,1,2011-08-18,Packaging quality problem,"""Both"" of Gloria Jean's ""Hazelnut"" and ""Vanill..."
473106,473107,B008FHUKE6,A17950SQVNAVOD,Scott,2,2,1,2011-08-18,Packaging quality problem,"""Both"" of Gloria Jean's ""Hazelnut"" and ""Vanill..."
245224,245225,B0029XDZKI,A17950SQVNAVOD,Scott,2,2,1,2011-08-18,Packaging quality problem,"""Both"" of Gloria Jean's ""Hazelnut"" and ""Vanill..."
425981,425982,B000TQEWM2,A17950SQVNAVOD,Scott,2,2,1,2011-08-18,Packaging quality problem,"""Both"" of Gloria Jean's ""Hazelnut"" and ""Vanill..."


In [16]:
# List of specific columns to check for duplicates
columns_to_check = ['ProductId', 'UserId', 'ProfileName', 'Summary', 'Text']

# Loop through each column and count duplicates
for col in columns_to_check:
    dup_count = df.duplicated(subset=[col]).sum()
    print(f"Duplicates in '{col}': {dup_count}")

Duplicates in 'ProductId': 494196
Duplicates in 'UserId': 312395
Duplicates in 'ProfileName': 350038
Duplicates in 'Summary': 272711
Duplicates in 'Text': 174875


In [17]:
# Group by text and filter to get groups where the size is greater than 1 (i.e., duplicates)
dup_text_groups = df.groupby('Text',observed=False).filter(lambda x: len(x) > 1)

# Sort the groups by 'textCat'
dup_text_groups_sorted = dup_text_groups.sort_values(by='Text')

# Print the duplicated rows, grouped by 'textCat'
dup_text_groups_sorted

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
257785,257786,B000KOWR8E,A142S4ZZF1FJ1X,Joseph E Brew,2,3,4,2010-10-09,Better Sweetener!,"""4C Totally Light"" is one of the very few ""sug..."
506745,506746,B000KOWR8Y,A142S4ZZF1FJ1X,Joseph E Brew,0,0,4,2010-10-09,4C Totally Light,"""4C Totally Light"" is one of the very few ""sug..."
107704,107705,B001F0RRTQ,A1R7E82MN0S8V3,DENNIS,0,0,5,2012-06-12,GREAT DOG TREAT,"""BUFFY"" LOOKS FORWARD TO HER ""TOY"" EVERY AFTER..."
418609,418610,B001F0RRU0,A1R7E82MN0S8V3,DENNIS,0,0,5,2012-06-12,GREAT DOG TREAT,"""BUFFY"" LOOKS FORWARD TO HER ""TOY"" EVERY AFTER..."
561246,561247,B001JU81ZG,A7FNPP1SMY97G,D. Hsu,4,6,1,2011-11-08,Buy this if you have NO taste buds!,"""Blends smooth and creamy for a sweet tasting ..."
...,...,...,...,...,...,...,...,...,...,...
48026,48027,B004SRH2B6,A13RF7W3A98FS0,utah2008,0,0,5,2011-10-01,great tasting,zico probably has the best taste of the coconu...
268751,268752,B004SRFYMK,A13RF7W3A98FS0,utah2008,0,0,5,2011-10-01,great tasting,zico probably has the best taste of the coconu...
292422,292423,B0022N49KU,A1R58LWNVV94NA,"Toni T. ""AboveAstar""",2,2,5,2011-12-20,"EXCELLENT Quality & Taste; Very, Very Versatil...","~ I ordered Farie's Finest Coconut powder, Map..."
167012,167013,B0022N49M8,A1R58LWNVV94NA,"Toni T. ""AboveAstar""",2,2,5,2011-12-20,"EXCELLENT Quality & Taste; Very, Very Versatil...","~ I ordered Farie's Finest Coconut powder, Map..."


In [18]:
# Now, filter within each group for rows where 'ProductId', 'UserId', 'ProfileName' are the same
dup_filtered = dup_text_groups.groupby(['Text','UserId','ProfileName','Summary'],observed=False).filter(lambda x: len(x) > 1)

# Sort the groups by 'textCat' and other relevant columns
dup_filtered_sorted = dup_filtered.sort_values(by=['Text','UserId','ProfileName','Summary'])

# Print the filtered and sorted duplicated rows
dup_filtered_sorted

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text
107704,107705,B001F0RRTQ,A1R7E82MN0S8V3,DENNIS,0,0,5,2012-06-12,GREAT DOG TREAT,"""BUFFY"" LOOKS FORWARD TO HER ""TOY"" EVERY AFTER..."
418609,418610,B001F0RRU0,A1R7E82MN0S8V3,DENNIS,0,0,5,2012-06-12,GREAT DOG TREAT,"""BUFFY"" LOOKS FORWARD TO HER ""TOY"" EVERY AFTER..."
330089,330090,B001OHX1ZY,A7FNPP1SMY97G,D. Hsu,4,6,1,2011-11-08,Buy this if you have NO taste buds!,"""Blends smooth and creamy for a sweet tasting ..."
561246,561247,B001JU81ZG,A7FNPP1SMY97G,D. Hsu,4,6,1,2011-11-08,Buy this if you have NO taste buds!,"""Blends smooth and creamy for a sweet tasting ..."
184223,184224,B006ZC3IHY,A17950SQVNAVOD,Scott,0,0,1,2011-08-18,Packaging quality problem,"""Both"" of Gloria Jean's ""Hazelnut"" and ""Vanill..."
...,...,...,...,...,...,...,...,...,...,...
268751,268752,B004SRFYMK,A13RF7W3A98FS0,utah2008,0,0,5,2011-10-01,great tasting,zico probably has the best taste of the coconu...
358777,358778,B003CIBPN8,A13RF7W3A98FS0,utah2008,0,0,5,2011-10-01,great tasting,zico probably has the best taste of the coconu...
113740,113741,B0022N26NM,A1R58LWNVV94NA,"Toni T. ""AboveAstar""",2,2,5,2011-12-20,"EXCELLENT Quality & Taste; Very, Very Versatil...","~ I ordered Farie's Finest Coconut powder, Map..."
167012,167013,B0022N49M8,A1R58LWNVV94NA,"Toni T. ""AboveAstar""",2,2,5,2011-12-20,"EXCELLENT Quality & Taste; Very, Very Versatil...","~ I ordered Farie's Finest Coconut powder, Map..."


### Step 5.1: Dropping duplicated rows

In [19]:
# Drop duplicates from the main DataFrame based on the specified columns
df_deduped = df.drop_duplicates(subset=['Text','UserId', 'ProfileName', 'Summary'])

# Assign it back to df if you want to overwrite the original
df = df_deduped

# Optional: check the shape after deduplication
print(f"Updated DataFrame shape: {df.shape}")

Updated DataFrame shape: (395016, 10)


In [20]:
# Group by 'Text' and filter to get groups where the size is greater than 1 (i.e., duplicates)
dup_textCat_groups = df.groupby('Text',observed=False).filter(lambda x: len(x) > 1)

# Now, filter within each group for rows where 'ProductId', 'UserId', 'ProfileName' are the same
dup_filtered = dup_textCat_groups.groupby(['Text','UserId','ProfileName','Summary'],observed=False).filter(lambda x: len(x) > 1)

# Sort the groups by 'textCat' and other relevant columns
dup_filtered_sorted = dup_filtered.sort_values(by=['Text','UserId','ProfileName','Summary'])

# Print the filtered and sorted duplicated rows
dup_filtered_sorted

,Id,ProductId,UserId,ProfileName,HelpfulnessNumerator,HelpfulnessDenominator,Score,Time,Summary,Text


## Sentiment Analysis
### Step 1: Combining Summary and Text columns to create list of all the words.

In [21]:
# Combine 'summary' and 'text' columns
combined_text = df['Summary'].fillna('') + ' ' + df['Text'].fillna('')

# Extract words using regex, convert to lowercase
all_words = []
for entry in combined_text:
    entry_words = re.findall(r'\b\w+\b', entry.lower())
    all_words.extend(entry_words)  # Add words to the list

# Print the number of words
print(f"Total words extracted: {len(all_words)}")

Total words extracted: 34365117


### Step 2: Removing Nonsense words and Stop words from the list

In [24]:
# Download the stopwords list (only once)
nltk.download('stopwords')

# Get the list of English stopwords
stop_words = set(stopwords.words('english'))

# Function to clean words
def clean_word(word):
    # Remove non-alphabetical characters and check if it's a valid word
    return re.match(r'^[a-zA-Z]+$', word) is not None and word not in stop_words

# Assuming 'all_words' contains all words
cleaned_words = [word for word in all_words if clean_word(word.lower())]

# Print first 20 cleaned words and total count
print(cleaned_words[:20])
print(f"Total cleaned words: {len(cleaned_words)}")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prarthana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['good', 'quality', 'dog', 'food', 'bought', 'several', 'vitality', 'canned', 'dog', 'food', 'products', 'found', 'good', 'quality', 'product', 'looks', 'like', 'stew', 'processed', 'meat']
Total cleaned words: 17164436


In [25]:
# Download NLTK words corpus
nltk.download('words')

[nltk_data] Downloading package words to /Users/prarthana/nltk_data...
[nltk_data]   Package words is already up-to-date!


True

In [26]:
# Download NLTK stopwords list (only once)
nltk.download('stopwords')

# Get stop words from NLTK
stop_words = set(stopwords.words('english'))

# Assuming cleaned_words is already defined from previous steps
# Filter out stop words from the cleaned words list
filtered_words = [word for word in all_words if word.lower() not in stop_words]

# Print the first 20 filtered words and total count of filtered words
print(filtered_words[:20])
print(f"Total filtered words: {len(filtered_words)}")

[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/prarthana/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['good', 'quality', 'dog', 'food', 'bought', 'several', 'vitality', 'canned', 'dog', 'food', 'products', 'found', 'good', 'quality', 'product', 'looks', 'like', 'stew', 'processed', 'meat']
Total filtered words: 17587203


### Step 3:  Correcting the mispelled words
Author's note: This takes a while( about 10 mins) to run so be patient.

In [27]:
# Initialize SymSpell with a max dictionary edit distance of 2
sym_spell = SymSpell(max_dictionary_edit_distance=2, prefix_length=7)

# Load the dictionary from NLTK or create your own (e.g., words corpus)
dictionary = set(words.words())


In [29]:
# Add words to SymSpell dictionary
for word in dictionary:
    sym_spell.create_dictionary_entry(word, 1)  # Frequency is set to 1

In [30]:
# Function to clean and correct words using SymSpell
def clean_word(word):
    # Remove non-alphabetical characters
    if not re.match(r'^[a-zA-Z]+$', word):
        return False
    
    # Correct spelling using SymSpell
    suggestions = sym_spell.lookup(word, Verbosity.CLOSEST, max_edit_distance=2)
    
    # If SymSpell found a valid suggestion, return the corrected word
    return suggestions[0].term if suggestions else None


In [31]:
# Clean the words list
cleaned_words = [word for word in filtered_words if clean_word(word.lower())]

# Print the first 20 cleaned words and total count
print(cleaned_words[:20])
print(f"Total cleaned words: {len(cleaned_words)}")

['good', 'quality', 'dog', 'food', 'bought', 'several', 'vitality', 'canned', 'dog', 'food', 'products', 'found', 'good', 'quality', 'product', 'looks', 'like', 'stew', 'processed', 'meat']
Total cleaned words: 17070744


In [32]:
# Print the first 20 cleaned words and total count
print(cleaned_words[:20])
print(f"Total cleaned words: {len(cleaned_words)}")

['good', 'quality', 'dog', 'food', 'bought', 'several', 'vitality', 'canned', 'dog', 'food', 'products', 'found', 'good', 'quality', 'product', 'looks', 'like', 'stew', 'processed', 'meat']
Total cleaned words: 17070744


### Note:
Creating another df of unique words for future use so we have frequency of words as well as list of unique words. We wouldnt have to run sentiment analysis on the repeated words as TextBlob uses a predefined dictionary (lexicon) of words where each word has an associated polarity (how positive/negative it is) and subjectivity (how opinionated it is).

In [ ]:
cleaned_words2 = list(set(cleaned_words))

# Print first 20 words and total count after removing duplicates
print(cleaned_words2[:20])
print(f"Total cleaned words after removing duplicates: {len(cleaned_words2)}")

### Step 4: Classifying words into 'positive', 'negative', and 'neutral'.

In [34]:
# Function to classify sentiment using TextBlob
def classify_sentiment_textblob(words_list):
    sentiment_data = []  # List to store word and sentiment pairs
    
    # Classify sentiment for each word in the list
    for word in words_list:
        polarity = TextBlob(word).sentiment.polarity
        if polarity > 0:
            sentiment = "Positive"
        elif polarity < 0:
            sentiment = "Negative"
        else:
            sentiment = "Neutral"
        
        sentiment_data.append([word, sentiment])
    
    # Convert to DataFrame for easier manipulation
    df_sentiment = pd.DataFrame(sentiment_data, columns=["Word", "Sentiment"])
    return df_sentiment

# Example: Classify sentiment for the cleaned words
df_sentiment = classify_sentiment_textblob(cleaned_words2)

# Display the resulting DataFrame
print(df_sentiment.head())  # Show first 5 rows of the DataFrame


          Word Sentiment
0      lilburn   Neutral
1    refluxers   Neutral
2   inheriting   Neutral
3  rewatchable   Neutral
4    stonhouse   Neutral


### Step 5: Categorizing words, frequency and sentiment to analyse the overall sentiment being observed

In [35]:
# Step 1: Count frequency of each word in cleaned_words
word_count = {}
for word in cleaned_words:
    word_count[word] = word_count.get(word, 0) + 1

# Create a DataFrame from word_count (Word and Count)
word_count_df = pd.DataFrame(list(word_count.items()), columns=["Word", "Count"])

# Step 2: Merge the word count DataFrame with the sentiment DataFrame
# We'll merge on the "Word" column to add sentiments to the word count DataFrame
df_merged = pd.merge(word_count_df, df_sentiment, on="Word", how="left")

# Step 3: Sort the merged DataFrame by count in descending order
df_merged_sorted = df_merged.sort_values(by="Count", ascending=False)

# Display the resulting merged and sorted DataFrame
print(df_merged_sorted.head())

        Word   Count Sentiment
211       br  449305   Neutral
12      like  183433   Neutral
0       good  175306  Positive
92     great  165454  Positive
10   product  135573   Neutral


### Step 6: Computing the Overall Sentiment using frequency as weights

In [36]:
# Map sentiment labels to numerical values
sentiment_map = {"Positive": 1, "Neutral": 0, "Negative": -1}
df_merged_sorted['SentimentValue'] = df_merged_sorted['Sentiment'].map(sentiment_map)

# Compute the weighted sentiment score
weighted_sentiment = (df_merged_sorted['SentimentValue'] * df_merged_sorted['Count']).sum() / df_merged_sorted['Count'].sum()

# Determine overall sentiment label
if weighted_sentiment > 0:
    overall_sentiment = "Positive"
elif weighted_sentiment < 0:
    overall_sentiment = "Negative"
else:
    overall_sentiment = "Neutral"

# Output the results
print(f"Weighted Sentiment Score: {weighted_sentiment}")
print(f"Overall Sentiment: {overall_sentiment}")

Weighted Sentiment Score: 0.08701425081414144
Overall Sentiment: Positive
